# About
* **Author**: Adil Rashitov
* **Created at**: 22.07.2021
* **Goal**: Solve linear programming OR problem

In [1]:
# Imports / Configs / Global vars

# Import of native python tools
import os
import json
from functools import reduce

# Import of base ML stack libs
import numpy as np
import sklearn as sc

# Multiprocessing for Mac / Linux
import platform
platform.system()
if platform.system() == 'Darwin':
    from multiprocess import Pool
else:
    from multiprocessing import Pool

# Visualization libraries
import plotly.express as px

# Logging configuraiton
import logging
logging.basicConfig(format='[ %(asctime)s ][ %(levelname)s ]: %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Ipython configs
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
display(HTML("<style>.container { width:100% !important; }</style>"))
InteractiveShell.ast_node_interactivity = 'all'

# Pandas configs
import pandas as pd
import geopandas as gpd
pd.options.display.max_rows = 350
pd.options.display.max_columns = 250

# Jupyter configs
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# GLOBAL VARS

In [33]:
!pip3 install ortools
!pip3 install statsmodels
from ortools.linear_solver import pywraplp

     |████████████████████████████████| 9.4 MB 780 kB/s eta 0:00:01     |████████████████▏               | 4.8 MB 780 kB/s eta 0:00:06
     |████████████████████████████████| 231 kB 10.7 MB/s eta 0:00:01


# Task 1 | Production planning

* **Goal**: Maximize the total profit
* **Change variables**: X - vector (Amount of products to produce for each product type)
* **Inputs**:
    * `D` - Demand matrix to produce single unit of product
    * `P` - Price of single unit produced product
    * `L` - Limits of raw materials supply

* **Constrains**:
    1. Material 1 must be in range `[0, 100]`
    1. Material 2 must be in range `[0, 150]`
    1. Material 3 must be in range `[0, 200]`


* **Steps**
    1. Data definition
    2. Objective function definition
    3. Constrains definition
    4. Summary before solving
    5. Solving
    6. Printing results

### 1. Data definition

In [3]:
# Demand of raw materials to produce single unit of good
D = np.array([
    [0, 3, 10],
    [5, 10, 10],
    [5, 3, 9],
    [4, 6, 3],
    [8, 2, 8],
    [5, 2, 10],
    [3, 2, 7],
])

 # Prices
P = np.array([100, 120, 135, 90, 125, 110, 105])

# Material supply limit
L = np.array([100, 150, 200])

X = [1, 1, 1, 1, 1, 1, 1]

solver = pywraplp.Solver.CreateSolver('GLOP')

### 2. set objective function

In [4]:
# OBJECTIVE: Maximize revenue of products

# 1. Definition of product array
x_vec = [
    solver.NumVar(0, solver.infinity(), f"Product: {x+1}")
    for x in range(D.shape[0])
]
x_vec = np.array(x_vec)

# 2. Maximization Price * quantity
solver.Maximize(x_vec @ P)

In [5]:
# Lets verify that solution is infeasible
solver.Solve() == solver.INFEASIBLE

True

Okay, as we see we defined our target however did't add constrains to make solution feasible

### Below are listed statuses that solver may return
![image](pictures/SOLVER_STATUSES.png)

### 3. Constrains definition

In [6]:
# x_vec @ D:
# How many raw materials are needed
# to produce a single unit of product
for x, lower, upper in zip(x_vec @ D, [0]*len(x_vec), L):
    _ = solver.Add((x >= lower) and (x <= upper))

### 4. Summary before solving

In [7]:
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

Number of variables = 7
Number of constraints = 3


### 5. Solving

In [11]:
solver.Solve() == solver.OPTIMAL

True

### 6. Printing results

In [12]:
solver.Objective().Value()

3404.4585987261144

In [16]:
pd.DataFrame([(x, x.solution_value()) for x in x_vec], columns=['produce_id', 'amount_to_produce'])

,produce_id,amount_to_produce
0,Product: 1,7.961783
1,Product: 2,0.000000
2,Product: 3,0.000000
3,Product: 4,17.834395
4,Product: 5,0.000000
5,Product: 6,0.000000
6,Product: 7,9.554140


# Task 2 | Finding β

![image](pictures/Task_2.png)

In [46]:
data = """
38	137
56	201
50	152
52	107
37	150
60	173
67	194
54	166
59	154
43	137
30	38
53	193
59	154
40	175
65	247
"""

data = list(filter(lambda x: x if x != '' else None, data.split('\n')))
x = pd.Series(map(lambda x: float(x.split('\t')[0]), data))
y = pd.Series(map(lambda x: float(x.split('\t')[1]), data))

In [47]:
import statsmodels.api as sm
x = sm.add_constant(x)
model = sm.OLS(y,x)
results = model.fit()


results.params

const    6.869444
0        2.981597
dtype: float64

In [39]:
duncan_prestige = sm.datasets.get_rdataset("Duncan", "carData")

In [41]:
duncan_prestige.data['income']

accountant            62
pilot                 72
architect             75
author                55
chemist               64
minister              21
professor             64
dentist               80
reporter              67
engineer              72
undertaker            42
lawyer                76
physician             76
welfare.worker        41
teacher               48
conductor             76
contractor            53
factory.owner         60
store.manager         42
banker                78
bookkeeper            29
mail.carrier          48
insurance.agent       55
store.clerk           29
carpenter             21
electrician           47
RR.engineer           81
machinist             36
auto.repairman        22
plumber               44
gas.stn.attendant     15
coal.miner             7
streetcar.motorman    42
taxi.driver            9
truck.driver          21
machine.operator      21
barber                16
bartender             16
shoe.shiner            9
cook                  14
